In [ ]:
import cv2, numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## Load Parameters

In [ ]:
from utils import read_json

params = read_json('parameters.json')

RESIZE_X = params['resize']['x']
RESIZE_Y = params['resize']['y']
ITEM_FOLDER = params['item_folder']

## Input Data
* Image of the bin
* List of items in the bin

In [ ]:
bin_stamp = '170405145336'
contents = ["Colgate_Toothbrush_4PK","Epsom_Salts","Duct_Tape",
            "Bath_Sponge","Crayons","Burts_Bees_Baby_Wipes"]

In [ ]:
bin_stamp = '170405145538'
contents = ["glue_sticks","tissue_box","laugh_out_loud_jokes",
            "toilet_brush","expo_eraser","table_cloth"]

In [ ]:
contents = [s.lower() for s in contents]

## Phase 1: Recognition by SIFT Features
### Compute Features in Bin

In [ ]:
from utils import imread_rgb, compute_sift

filename_bin = 'bin/' + bin_stamp + '.png'
image_bin = imread_rgb(filename_bin)
(kp_bin, des_bin) = compute_sift(image_bin)

### Match Bin and Items Features

In [ ]:
from utils import match_items

items = list(contents)
item_d, recognised_items, mask_items = match_items(image_bin, kp_bin, des_bin, items)

### Not recognised items

In [ ]:
items = [s for s in contents if s not in recognised_items]
items

In [ ]:
kernel = np.ones((3,3),np.uint8)
mask_items = 255 - cv2.dilate(mask_items,kernel,iterations = 5)
plt.imshow(mask_items,cmap='gray'), plt.axis('off');

## Phase 2: segment bin by depth from top to bottom

In [ ]:
from utils import imread_gray
filename_bin = 'bin/' + bin_stamp + '.pgm'
image_depth = imread_gray(filename_bin)
plt.imshow(image_depth,cmap='gray'); plt.axis('off');

In [ ]:
from utils import fill_holes

image_depth = fill_holes(image_depth)
plt.imshow(image_depth,cmap='gray');
(np.min(image_depth), np.max(image_depth))

In [ ]:
import glob
import json
dc_list = []
for item in items:
    folder = 'Training_items/' + item + '/'
    files = glob.glob(folder + '*_dc.json')
    for filename in files:
        with open(filename) as data_file:
            dominant_colors = json.load(data_file)
        dc_list.append((filename,dominant_colors))

In [ ]:
from utils import compute_colors, calc_EMD2

depth_int = 4

min_depth = int(np.min(image_depth))
max_depth = int(np.max(image_depth))
min_emd = {}
for thresh_depth in range(min_depth, max_depth, depth_int/2):
    top_obj = cv2.inRange(image_depth, thresh_depth, thresh_depth+depth_int)
    top_obj = cv2.bitwise_and(top_obj, top_obj, mask=mask_items)
    image_top = cv2.bitwise_and(image_bin,image_bin,mask=top_obj)
    plt.imshow(image_top); plt.axis('off'); plt.title(str(thresh_depth)); plt.show();
    (cnt,_) = cv2.findContours(top_obj,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for c in cnt:
        a = cv2.contourArea(c)
        if a > 1000 and a < 40000:
            item_mask = np.zeros(top_obj.shape,dtype='uint8')
            cv2.drawContours(item_mask,[c],-1,(255,),-1)
            #image_top = cv2.bitwise_and(image_bin,image_bin,mask=item_mask)
            h_obj, cc_obj = compute_colors(image_bin, item_mask)
            for name, dc in dc_list:
                short_name = name[15:-8]
                h_ref = dc['hist']
                cc_ref = dc['cluster_centers']
                emd = calc_EMD2(h_obj,cc_obj,h_ref,cc_ref)
                try:
                    if emd < min_emd[short_name][0]:
                        min_emd[short_name] = (emd, c)
                except KeyError:
                    min_emd[short_name] = (emd, c)

In [ ]:
for name, cd in min_emd.iteritems():
    if cd[0] < 100:
        item_mask = np.zeros(top_obj.shape,dtype='uint8')
        cv2.drawContours(item_mask,[cd[1]],-1,(255,),-1)
        image_top = cv2.bitwise_and(image_bin,image_bin,mask=item_mask)
        plt.imshow(image_top); plt.axis('off'); plt.title(name + ': ' + str(cd[0])); plt.show();